# Aula TP - 15/Mar/2022

#### Grupo 8
- Melânia Pereira
- Duarte Oliveira
- Paulo Pereira

# Parte V

### Pergunta P.V.1.1

As adições pedidas foram tidas em conta já na resolução do problema na ficha anterior.  O programa desenvolvido permite ao utilizador inserir uma chave de um qualquer tamanho, sendo que esta é depois passada a uma função KDF onde é usado um sal e que, a partir da chave inserida pelo utilizador, deriva uma nova chave, com o tamanho correto de 32 bytes. Optamos pela função `PBKDF2` pois parece ser a mais indicada para a derivação de uma chave com base numa expécie de 'password' que, neste caso, é a chave inserida pelo utilizador. <br>
Decidimos adicionar estes passos na resolução do exercício da aula anterior por percebermos exatamente que o ChaCha20 requer uma chave com um tamanho específico e, em vez de fazermos esse pedido ao utilizador, decidimos procurar uma forma de assegurar isso. Então, percebemos que a forma mais fácil e imediata seria passar a chave do utilizador a uma função de derivação de chaves que derivasse uma chave do tamanho certo (32 bytes) a partir da chave do utilizador.

In [ ]:
import sys
import json
from base64 import b64encode
from base64 import b64decode
from Crypto.Cipher import ChaCha20
from Crypto.Random import get_random_bytes
from Crypto.Protocol.KDF import PBKDF2
from Crypto.Hash import SHA512
from Crypto.Random import get_random_bytes


def derive_key(password):
    salt=b'z\x95\x81\x97,RrP\xde]4\x81\xe4\xf2\x91\x9c'
    key = PBKDF2(password, salt, 32, count=1000000, hmac_hash_module=SHA512)
    return key


def cipher(key,data):
    nonce_rfc7539 = get_random_bytes(12)

    cipher = ChaCha20.new(key=key, nonce=nonce_rfc7539)
    ciphertext = cipher.encrypt(data)
    
    ct = b64encode(ciphertext).decode('utf-8')
    ciphertext = b64encode(ciphertext).decode('utf-8')
    nonce_rfc7539 = b64encode(nonce_rfc7539).decode('utf-8')

    result = json.dumps({'nonce':nonce_rfc7539, 'ciphertext':ct})
    
    return result


def decipher(key,data):
    try:
        b64 = json.loads(data)
        nonce = b64decode(b64['nonce'])
        ciphertext = b64decode(b64['ciphertext'])

        cipher = ChaCha20.new(key=key, nonce=nonce)
        plaintext = cipher.decrypt(ciphertext)
    except (ValueError, KeyError):
        print("Incorrect decryption")

    return plaintext.decode("utf-8")


# CLI

args = len(sys.argv)
if args>5 or args< 4:
    print("Not enough arguments")
else:
    if args < 5:
        # chacha.py encrypt/decrypt input.txt output.txt 
        while True:
            try:
                password = input("Please enter the key: ")
                cd = sys.argv[1]
                inputfile = sys.argv[2]
                outputfile = sys.argv[3]
            except ValueError:
                continue
            else:
                break
    if args == 5:
        # chacha.py encrypt/decrypt input.txt output.txt password
        cd = sys.argv[1]
        inputfile = str(sys.argv[2])
        outputfile = str(sys.argv[3])
        password = str(sys.argv[4])
    

    
    # managing input file and getting the data from it
    fin = open(inputfile)
    data = fin.read()
    fin.close()

    # managing output file and getting it ready to receive data  
    fout = open(outputfile,'w')

    # key derivation to get the 32 bytes for the chacha algorithm
    key = derive_key(password)

    if cd != "encrypt" and cd !="decrypt":
        print("unknown command: must spell correctly encrypt/decrypt !\n")
    else:
        if cd == "encrypt":
            print("encryption chosen")
            text = cipher(key,bytes(data, encoding='utf8'))
            print(text)
            fout.write(text)
            fout.close()
            
        if cd =="decrypt":
            print("decryption chosen")
            text = decipher(key,data)
            fout.write(text)
            print(text)
            fout.close()

### Pergunta P.V.1.2

1. A razão deve-se ao facto de ser vulnerável a ataques de dicionário, onde o atacante cataloga todo o espaço de chaves, ou por testagem interativa das várias possibilidades. A consideração de fatores aleatórios como o salt e o IV podem dificultar tais ataques. No entanto, deve-se sempre optar por uma hash utilizando uma PBKDF. As funções de hash irreversíveis como md5, sha1, sha256, sha512 são facilmente revertidas por adversários.


2. Para encontrar a palavra-passe representada pela hash fornecida e sabendo que esta faz parte do Top200 fornecido, começamos por obter a lista das 200 passwords nesse mesmo top e calcular a representação hexadecimal de cada uma delas, comparando com a fornecida. Trata-se de um ataque por _brute force_, pois são testadas todas as possibilidades até chegar à correta. <br>
A baixo encontra-se o código desenvolvido. <br>
*NOTA*: Pesquisando na internet pela representação fornecida, é imediato perceber de qual palavra-passe se trata.

In [2]:
#2.
from hashlib import sha256

top200 = ['123456','123456789','picture1','password','12345678','111111','123123','12345','1234567890','senha','1234567','qwerty','abc123','Million2','000000','1234','iloveyou','aaron431','password1','qqww1122','123','omgpop','123321','654321','qwertyuiop','qwer123456','123456a','a123456','666666','asdfghjkl','ashley','987654321','unknown','zxcvbnm','112233','chatbooks','20100728','123123123','princess','jacket025','evite','123abc','123qwe','sunshine','121212','dragon','1q2w3e4r','5201314','159753','123456789','pokemon','qwerty123','Bangbang123','jobandtalent','monkey','1qaz2wsx','abcd1234','default','aaaaaa','soccer','123654','ohmnamah23','12345678910','zing','shadow','102030','11111111','asdfgh','147258369','qazwsx','qwe123','michael','football','baseball','1q2w3e4r5t','party','daniel','asdasd','222222','myspace1','asd123','555555','a123456789','888888','7777777','fuckyou','1234qwer','superman','147258','999999','159357','love123','tigger','purple','samantha','charlie','babygirl','88888888','jordan23','789456123','jordan','anhyeuem','killer','basketball','michelle','1q2w3e','lol123','qwerty1','789456','6655321','nicole','naruto','master','chocolate','maggie','computer','hannah','jessica','123456789a','password123','hunter','686584','iloveyou1','0987654321','justin','cookie','hello','blink182','andrew','25251325','love','987654','bailey','princess1','0123456','101010','12341234','a801016','1111','1111111','anthony','yugioh','fuckyou1','amanda','asdf1234','trustno1','butterfly','x4ivygA51F','iloveu','batman','starwars','summer','michael1','00000000','lovely','jakcgt333','buster','jennifer','babygirl1','family','456789','azerty','andrea','q1w2e3r4','qwer1234','hello123','10203','matthew','pepper','12345a','letmein','joshua','131313','123456b','madison','Sample123','777777','football1','jesus1','taylor','b123456','whatever','welcome','ginger','flower','333333','1111111111','robert','samsung','a12345','loveme','gabriel','alexander','cheese','passw0rd','142536','peanut','11223344','thomas','angel1']

for password in top200:
    if sha256(password.encode('utf-8')).hexdigest() == '96cae35ce8a9b0244178bf28e4966c2ce1b8385723a96a6b838858cdd6ca0a1e':
        print(password)

123123


### Pergunta P.V.1.3

1. openssl dgst -sha1 -hmac \<chave para o hmac\> \<caminho para a diertoria\>/*

2. Teria de efetuar a operação novamente e comparar o HMAC-SHA1 de cada ficheiro com o obtido anteriormente, se este for diferente, significa que o ficheiro foi alterado.


# Parte VI

### Pergunta P.VI.1.1

In [1]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.fernet import Fernet
import base64


# Alice define g e tamanho de p
g = int(input("Alice chooses g: "))
key_size = int(input("Alice choose the key_size: "))

# Alice inicializa parâmetros do Diffie-Hellman
print(f"Generating parameters for Alice with g={g} and key size of {key_size}...")
alice_parameters = dh.generate_parameters(generator=g, key_size=key_size) 
print("Parameters generated:", alice_parameters.parameter_numbers)
# Alice obtém p
p = alice_parameters.parameter_numbers().p
print("Alice gets p:",p)
# Alice gera a e ga
print("Generating a (private key) and g^a (public key) for Alice...")
a = alice_parameters.generate_private_key()
ga = a.public_key()
print("Alice has g^a =", ga)
#
#Alice envia p, g e ga a Bob
print()
print()
print("Sending p, g and g^a from Alice to Bob ...")
print()
print()
#
print()
print()
print("Bob is receiving p, g and g^a from Alice ...")
print()
print()
print("Initializing Bob's DH parameter with p and g received from Alice...")
# Bob inicializa parâmetro do Diffie-Hellman e gera b e gb
bob_parameters = dh.DHParameterNumbers(p=p, g=g).parameters()
print("Calculating Bob's b (private key) and g^b (public key)...")
b = bob_parameters.generate_private_key()
gb = b.public_key()
print("Bob has g^b = ", gb)
# Bob obtém a shared key, a partir de gb e ga
print("Bob is calculating the shared key with Alice")
bob_sharedkey = b.exchange(ga)
print("Bob has the shared key with Alice!")

print("Bob is deriving a key from the shared key...")
bob_derivedkey = HKDF(
    algorithm=hashes.SHA256(),
    length=32,
    salt=None,
    info=b'handshake data',
).derive(bob_sharedkey)

print()
print("Done!")

#
# Bob envia gb a Alice
print()
print("Sending g^b from Bob to Alice ...")
print()
#
# Alice obtém a shared key a partir de ga e gb
print()
print("Alice is receiving g^b from Bob ...")
print()
print("Alice is calculating the shared key with Bob ...")
print()
alice_sharedkey = a.exchange(gb)
print("Alice has the shared key with Bob!")

print("Alice is deriving a key from the shared key...")
alice_derivedkey = HKDF(
    algorithm=hashes.SHA256(),
    length=32,
    salt=None,
    info=b'handshake data',
).derive(alice_sharedkey)

print()
print("Done!")

alice_message = input("Now Alice wants to send Bob the following message: ")

print("Alice is encrypting the message ...")
alice_derivedkey64 = base64.urlsafe_b64encode(alice_derivedkey)
alice_fernet = Fernet(alice_derivedkey64)
alice_encryptedtext = alice_fernet.encrypt(alice_message.encode('utf-8'))
print("Alice is sending to Bob the encrypted message:", alice_encryptedtext)
print()
print()


print()
print()
print("Bob is receiving a message from Alice ...")
print()
print()

print("Bob is decrypting the message ...")
bob_derivedkey64 = base64.urlsafe_b64encode(bob_derivedkey)
bob_fernet = Fernet(bob_derivedkey64)
bob_decryptedtext = bob_fernet.decrypt(alice_encryptedtext).decode('utf-8')

print("Bob has decrypted the message:", bob_decryptedtext)



Generating parameters for Alice with g=2 and key size of 1024...
Parameters generated: <bound method _DHParameters.parameter_numbers of <cryptography.hazmat.backends.openssl.dh._DHParameters object at 0x103c422b0>>
Alice gets p: 128782076396648316789148531965594392347856152941654863083164113210987403985318836691010469033598908396982939309431846507317815451494756858376868746488363979667249075676946741199509596758566988694792976838492250911566225306179791124841939283566725759379052885828947821136788040911561072406336984081748782938403
Generating a (private key) and g^a (public key) for Alice...
Alice has g^a = <cryptography.hazmat.backends.openssl.dh._DHPublicKey object at 0x108801f70>


Sending p, g and g^a from Alice to Bob ...




Bob is receiving p, g and g^a from Alice ...


Initializing Bob's DH parameter with p and g received from Alice...
Calculating Bob's b (private key) and g^b (public key)...
Bob has g^b =  <cryptography.hazmat.backends.openssl.dh._DHPublicKey object at 0x1086